# Function 5

## Function Description

You’re tasked with optimising a four-variable black-box function that represents the yield of a chemical process in a factory. The function is typically unimodal, with a single peak where yield is maximised. 

Your goal is to find the optimal combination of chemical inputs that delivers the highest possible yield, using systematic exploration and optimisation methods.

## Libraries

In [3]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel

## Data

In [5]:
# Initialize the dataset (Function 5)
df_init = pd.DataFrame({
    "x1": [0.191447084, 0.758652949, 0.438349873, 0.706050834, 0.836477993,
           0.68343225, 0.55362148, 0.352356269, 0.153785706, 0.463442267,
           0.677491148, 0.583973412, 0.306888719, 0.511141775, 0.438933376,
           0.224189023, 0.725261723, 0.35548161, 0.119879226, 0.12688467],

    "x2": [0.038193371, 0.536517738, 0.804339705, 0.534191961, 0.193609647,
           0.118662642, 0.667349979, 0.322241532, 0.72938169, 0.63002451,
           0.358509507, 0.147242646, 0.316878127, 0.817956997, 0.774091762,
           0.84648049, 0.479870486, 0.639619367, 0.862540306, 0.153429621],

    "x3": [0.607417811, 0.656000382, 0.210245266, 0.264243345, 0.663892697,
           0.82904591, 0.323805819, 0.116979368, 0.422598437, 0.107906456,
           0.479592224, 0.348097462, 0.622634481, 0.728710418, 0.378167086,
           0.87948418, 0.088946843, 0.417617679, 0.643331326, 0.770162188],

    "x4": [0.414584137, 0.360341553, 0.151294816, 0.482087549, 0.785648883,
           0.567576606, 0.814869754, 0.473112522, 0.443074166, 0.957643899,
           0.072880481, 0.428614651, 0.095399058, 0.112353623, 0.933696207,
           0.878515684, 0.75976022, 0.12260384, 0.849803829, 0.190518105],

    "y": [64.44343986, 18.3013796, 0.112939795, 4.210898129, 258.3705254,
          78.43438889, 57.57153693, 109.5718756, 8.847991759, 233.2236102,
          24.42308831, 64.42014682, 63.47671579, 79.72912993, 355.8068178,
          1088.859618, 28.86675164, 45.18157035, 431.6127568, 9.972331895]
})
new_data = [
    (0.423856, 0.892394, 0.987243, 0.956672, 2756.03124986405),  # week 1
    (0.887599, 0.988835, 0.968898, 0.982824, 5960.15468419458),  # week 2
    (0.973697, 0.980505, 0.882116, 0.973561, 5705.04089933277),  # week 3
    (0.998438, 0.977165, 0.989597, 0.83695, 5813.33691891979),  # week 4
    (0.100969, 0.012860, 0.041430, 0.064078, 162.770532298688),  # week 5
    (0.013728, 0.097174, 0.974985, 0.973978, 1231.53744271913),  # week 6
    (0.936747, 0.995745, 0.985364, 0.832010, 5119.09857262543),  # week 7
    (0.963530, 0.998848, 0.908314, 0.996204, 6502.61433947562),  # week 8
    (0.962201, 0.973217, 0.947207, 0.925597, 5574.60840558671),  # week 9
    (0.955383, 0.988362, 0.971563, 0.988867, 6982.05890328975),  # week 10
    (0.969758, 0.984378, 0.965564, 0.985687, 6990.71552177193),  # week 11
    (0.986930, 0.987995, 0.885717, 0.978686, 6135.61293364942),  # week 12
    (0.978144, 0.957645, 0.984103, 0.992869, 7120.02004390597),  # week 13
]
df_new = pd.DataFrame(new_data, columns=["x1", "x2", "x3", "x4", "y"])
df_all = pd.concat([df_init, df_new], ignore_index=True)
# Extract input (X) and output (y)
X_check = df_all[["x1", "x2", "x3", "x4"]].values  # shape (20, 4)
y_check = df_all["y"].values.reshape(-1, 1)  # shape (20, 1)

print("Dataset shape:", X_check.shape, y_check.shape)
print(df_all.tail())

# For later use in model training
X_init = df_all[["x1", "x2", "x3", "x4"]].to_numpy()
y_raw = df_all["y"].to_numpy()

Dataset shape: (33, 4) (33, 1)
          x1        x2        x3        x4            y
28  0.962201  0.973217  0.947207  0.925597  5574.608406
29  0.955383  0.988362  0.971563  0.988867  6982.058903
30  0.969758  0.984378  0.965564  0.985687  6990.715522
31  0.986930  0.987995  0.885717  0.978686  6135.612934
32  0.978144  0.957645  0.984103  0.992869  7120.020044


## Optimisation Model

In [7]:
# --- Adjustable parameters ---
n_candidates = 30000  # number of random candidate points to explore
nu = 2.5  # smoothness parameter for Matern kernel
noise_level = 1.0  # assumed noise (for WhiteKernel)
length_scale = 0.3  # initial length scale for Matern
beta = 1.0  # exploration parameter for UCB (higher = more exploration)
random_state = 42  # reproducibility

# --- Define kernel and GP model ---
kernel = ConstantKernel(1.0, (1e-2, 1e2)) * Matern(length_scale=length_scale, nu=nu) + WhiteKernel(
    noise_level=noise_level)
gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True, random_state=random_state)

# --- Fit GP to initial data ---
gp.fit(X_init, y_raw)

# --- Generate candidate points uniformly in [0,1]^4 ---
X_candidates = np.random.rand(n_candidates, 4)

# --- Predict mean and std for each candidate ---
mean, std = gp.predict(X_candidates, return_std=True)

# --- Convert mean/std back to original y scale ---
mean_orig = mean * np.std(y_raw) + np.mean(y_raw)
std_orig = std * np.std(y_raw)

# --- Compute UCB acquisition function (in original scale) ---
ucb = mean_orig + beta * std_orig  # for maximization

# --- Get top 5 candidates ---
top_idx = np.argsort(ucb)[-5:][::-1]
top_candidates = X_candidates[top_idx]
top_ucb_values = ucb[top_idx]
top_pred_y = mean_orig[top_idx]

# --- Display results ---
df_top = pd.DataFrame(top_candidates, columns=["x1", "x2", "x3", "x4"])
df_top["Pred_y"] = top_pred_y
df_top["UCB_value"] = top_ucb_values

print("\nTop 5 candidate points (highest UCB):")
print(df_top)
print("\nBest guess (highest UCB):")
print(df_top.iloc[0])


Top 5 candidate points (highest UCB):
         x1        x2        x3        x4        Pred_y     UCB_value
0  0.992174  0.941115  0.986583  0.991077  1.920145e+07  1.986965e+07
1  0.966306  0.846852  0.998453  0.998350  1.790992e+07  1.883554e+07
2  0.952479  0.858325  0.958943  0.999545  1.694653e+07  1.779353e+07
3  0.989582  0.771146  0.977079  0.976208  1.630326e+07  1.744174e+07
4  0.965887  0.980549  0.895398  0.969660  1.655587e+07  1.715557e+07

Best guess (highest UCB):
x1           9.921736e-01
x2           9.411153e-01
x3           9.865832e-01
x4           9.910774e-01
Pred_y       1.920145e+07
UCB_value    1.986965e+07
Name: 0, dtype: float64
